# Hazelcast Python Client SQL Support with Hazelcast Viridian

This notebook demonstrates the SQL support of Hazelcast Python Client. Hazelcast provides in-depth SQL support for Map data structures kept in Hazelcast Clusters. Using Hazelcast SQL, you can create mappings between your data and a Map and execute SQL queries against the Map. This support provides fast in-memory computing using SQL engine without writing complex functions that iterate through your maps.</br></br>
Through this tutorial, you can either connect to your local cluster or Hazelcast Viridian. In this Google Collaboratory version, we will use Hazelcast Viridian as our cluster provider to not worry about setup or installation. Hazelcast Viridian offers free registration with a 2GB storage area. You can register using the following link and create a cluster in a few minutes: https://viridian.hazelcast.cloud/.</br></br> 

If you want to run this notebook on Jupyter locally, you can download the notebook file from our GitHub repo: https://github.com/hazelcast/hazelcast-python-client/tree/master/examples </br></br> If you have any issues, you can ask your questions in Hazelcast Community channels, find links here: https://hazelcast.com/dev-community/ also feel free to reach out at developer-relations@hazelcast.com 

![Tux, the Linux mascot](https://media-exp1.licdn.com/dms/image/C561BAQEDzMqseo-qiA/company-background_10000/0/1626235318986?e=2147483647&v=beta&t=9QQp_S3AuK8-1nMqsSTlL0xy2_JpkNQrzFJ7D_SLUYU)


You can quickly create a new account in Hazelcast Viridian and create a cluster. After creating the cluster, we need your cluster name, discovery token and SSL password token to connect from this notebook. You can find these tokens from **Connect Client > Advanced Setup** tab in the Hazelcast Viridian dasboard. Also, please download certificate ZIP file by clicking "Download keystore file", we will need SSL certificate files to establish a secure connection with our cluster.
</br></br> <img src="https://media.giphy.com/media/vs7z8uXBd6v9044ZTL/giphy.gif" alt="this slowpoke moves"  width="600" /> 

In [ ]:
#@title Hazelcast Viridian Authentication Tokens

#@markdown Run this cell and paste your Viridian tokens to textboxes that will show up.
#@markdown It will ask for your cluster name, discovery token and keystore password. 
#@markdown Also, a file upload button will be showed up for you to upload ZIP file that contains your certificates.  

import zipfile
import random
import string
import tkinter as tk
import tempfile
from tkinter import filedialog
from getpass import getpass

CLUSTER_NAME = getpass('Your cluster name: ')
DISCOVERY_TOKEN = getpass('Your discovery token: ')
SSL_PASSWORD = getpass('Your keystore/trustore password token: ')
file_dir = ""

root = tk.Tk()
root.withdraw()
file_path = filedialog.askopenfilename()
zf = zipfile.ZipFile(file_path)
curr = !pwd
file_dir = curr[0]+'/hzkeys-'+''.join(random.choice(string.ascii_lowercase) for i in range(10))
zf.extractall(file_dir)

CA_PATH  = file_dir + "/ca.pem" 
CERT_PATH= file_dir + "/cert.pem"
KEY_PATH = file_dir + "/key.pem"
print("Your tokens and certificates successfully uploaded.")

Instead of hard-coding data entries to our Map, we will use an API source to pull large amount of data to our cluster. We decided to use a movie API named **The Movie Database (TMDB)** which returns in-depth movie and actor data for us as JSON objects. To be able to send a request to this API, you will need an API key. You can create it in 1-2 minutes from their website. Note that you need to create an account first and then go to **Settings > API > Create New Key**. It may ask for some information about your project; providing short descriptions like "Experimenting API requests" is enough.

https://www.themoviedb.org/signup

In [ ]:
#@title Connect to API
#@markdown Run this cell from top-left corner and paste your API key to the textbox.

API_KEY = getpass('Your API key: ')

Now, we have ready data source and data storage and now it's turn of Python client. We need to install Hazelcast Python client to use the functionality of Hazelcast from our Python application. You can embed Hazelcast inside your program by simply inserting a module, `hazelcast-python-client`, and utilize all the methods using this package.

# Install Hazelcast Client

In [ ]:
%pip install hazelcast-python-client

# Imports

In [ ]:
import hazelcast
import json
import requests

# Connect To Hazelcast Cluster

We are ready to connect our cluster from Python program. Hazelcast uses config settings to find your cluster. We are providing our Viridian tokens for configuration to connect Hazelcast Cloud. After seeing the `Connection successful.` message, we can create mappings for our data. 

Reminder: If you want to connect to a local cluster in Jupyter Notebook, you should download the notebook file and remove the Viridian configuration options inside `hazelcast.HazelcastClient()` functional call.

In [ ]:
hazelcast.discovery.HazelcastCloudDiscovery._CLOUD_URL_BASE = "api.viridian.hazelcast.cloud"
client = hazelcast.HazelcastClient(
    cluster_name=CLUSTER_NAME,
    cloud_discovery_token=DISCOVERY_TOKEN,
    statistics_enabled=True,
    ssl_enabled=True,
    ssl_cafile=CA_PATH,
    ssl_certfile=CERT_PATH,
    ssl_keyfile=KEY_PATH,
    ssl_password=SSL_PASSWORD
)

print("Connection successful. You are connected to your Viridian %s cluster." % CLUSTER_NAME)

# Create Mapping between your data and Map

Here is the critical part. We will insert the data from TMDB API to our Hazelcast Cluster. You can do this in two way. Firstly, you can get a map by calling `hazelcast.get_map()` method and use `put()`, `remove()` and other methods of the returned distributed map object. Secondly you can create a mapping for your data schema using SQL syntax and it directly creates both map and mapping. We preferred second way to show SQL functions in this notebook. 
</br></br>In the mapping query, you should sepicfy the fields of data schema. It can be both primitive types or complex types. Using this mapping, we will execute SQL queries on our maps. You can select any column you want to map; not all fields are mandatory. For more information about mapping, you can visit https://docs.hazelcast.com/hazelcast/5.1/sql/mapping-to-maps
</br></br>
Now, execute following cells to create mappings to execute SQL queries against your data.

In [ ]:
query = """
    CREATE OR REPLACE MAPPING movies (
        __key VARCHAR, 
        id INT,
        original_title VARCHAR,
        popularity DECIMAL,
        release_date VARCHAR,
        vote_average DECIMAL,
        vote_count INT
    )
    TYPE IMap OPTIONS(
        'keyFormat' = 'varchar',
        'valueFormat' = 'json-flat'
    )
"""
client.sql.execute(query).result()
print("Mapping for 'movies' has been created.")

In [ ]:
query = """
    CREATE OR REPLACE MAPPING casts (
        __key VARCHAR,
        movie_id INT,
        actor_id INT
    )
    TYPE IMap OPTIONS(
        'keyFormat' = 'varchar',
        'valueFormat' = 'json-flat'
    )
"""
client.sql.execute(query).result()
print("Mapping for 'cast' has been created.")

In [ ]:
query = """
    CREATE OR REPLACE MAPPING actors (
        __key VARCHAR,
        gender INT,
        id INT,
        name VARCHAR,
        popularity DECIMAL
    )
    TYPE IMap OPTIONS(
        'keyFormat' = 'varchar',
        'valueFormat' = 'json-flat'
    )
"""
client.sql.execute(query).result()
print("Mapping for 'actors' has been created.")

In [ ]:
query = """
    CREATE OR REPLACE MAPPING reviews (
        __key VARCHAR,
        movie_id INT,
        author VARCHAR,
        rating VARCHAR,
        content VARCHAR
    )
    TYPE IMap OPTIONS(
        'keyFormat' = 'varchar',
        'valueFormat' = 'json-flat'
    )
"""
client.sql.execute(query).result()
print("Mapping for reviews has been created.")

You can control your mappings by opening Management Center for your cluster. Select your cluster from your Viridian dashboard and then click to "Management Center". It will open the Management Center in a new tab, you can inspect your cluster properties from this page. Click to "SQL Browser" at the top-right of the page to open SQL prompt. 

<img src="https://media.giphy.com/media/3Htnl3mItmpvcdblDr/giphy.gif" alt="this slowpoke moves"  width="600" />

# Load Data from API

Now, we are ready to pull our data from the TMDB API and load them to our Hazelcast Cluster. To do that, we need to send requests to our API provider. Following cells send API requests and insert results into appropriate maps.

**This step may take time, depending on your connection speed and server latency. As long as there is no error and an `[*]` is present near to cell, do not worry.**

In [ ]:
key = 1
for i in range(1,25):
    data = requests.get("https://api.themoviedb.org/3/movie/top_rated?api_key=%s&language=en-US&page=%d" % (API_KEY, i)).json()
    for m in data['results']:
      query = """SINK INTO movies(__key, id, original_title, popularity, release_date, vote_average, vote_count)
                 VALUES(\'%s\', %d, \'%s\', %f, \'%s\', %f, %d)""" % (
                     key, 
                     m["id"], 
                     m["title"].replace("'","''"), 
                     m["popularity"], 
                     m["release_date"], 
                     m["vote_average"], 
                     m["vote_count"])
      client.sql.execute(query)
      key += 1
print("Movie data has been loaded.")

In [ ]:
key = 0; cast_key = 0;
for i in range(1,50):
    data = requests.get("https://api.themoviedb.org/3/person/popular?api_key=%s&language=en-US&page=%d" % (API_KEY,i)).json()
    for a in data['results']:

        query = """SINK INTO actors(__key, gender, id, name, popularity)
                   VALUES(\'%s\', %d, %d, \'%s\', %f)""" % (
                       key, 
                       a["gender"], 
                       a["id"],
                       a["name"],
                       a["popularity"])
        client.sql.execute(query)
        key += 1
        
        for c in a['known_for']:
            if c['media_type'] == 'movie':
                query = """SINK INTO casts(__key, movie_id, actor_id)
                          VALUES(\'%s\', %d, %d)""" % (
                              cast_key,
                              c['id'],
                              a['id'])
                client.sql.execute(query)
                cast_key += 1
print("Actor and Cast data has been loaded.")

In [ ]:
key = 0

with client.sql.execute("SELECT id FROM movies WHERE __key<\'%s\'"% 200).result() as result:
    for row in result:
      id = row['id']

      url = "https://api.themoviedb.org/3/movie/%d/reviews?api_key=%s&language=en-US&page=1" % (id, API_KEY)
      data = requests.get(url).json()

      for r in data['results']:

        query = """SINK INTO reviews(__key, movie_id, author, rating, content)
                   VALUES(\'%s\', %d, \'%s\', \'%s\', \'%s\')""" % (
                       key,
                       id, 
                       r['author_details']['username'], 
                       r['author_details']['rating'], 
                       r['content'].replace("'","''"))

        client.sql.execute(query).result()
        key += 1

print("Review data has been loaded.")

After the loading data operations, there should be approximately </br>
*   900 entry in `actors` map
*   2400 entry in `casts` map
*   500 entry in `movies` map
*   300 entry in `reviews` map</br>

You can control your Map size and content from the Management Center.

<img src="https://media.giphy.com/media/TuomuvI4fPUCRzRjkO/giphy.gif" alt="this slowpoke moves"  width="600" />

# Fun part: SQL queries 

In the following examples, we will observe different SQL funtions supported by Hazelcast and execute them to process our data quickly. You can edit the queries directly and experiment SQL support hands-on.  Fasten the belts!

## Movies released before 2015 and high vote average with cote count > 20K 

In this first query, we will use essential features of SQL, like the `WHERE` clause. We have some filters to select movies among 4K entries, like `vote_count`, `vote_average`, and `release_date`. You can use `AS` keyword to specify the column name of result. Without Hazelcast, typically, we need to iterate over our map and check whether the current entry fits into our definition or not. Also, this approach requires an additional structure to store our selected entries. With the inclusive SQL support of Hazelcast, we can translate our filters to a SQL query and execute it on our map. 

In [ ]:
query = """
    SELECT m.original_title AS name, m.vote_average AS average, m.vote_count AS cont
    FROM movies m
    WHERE m.vote_count > 20000 AND m.vote_average > 7 AND m.release_date < '2015-01-01'
    ORDER BY m.popularity DESC
"""

result = client.sql.execute(query).result()

for row in result:
    print('%s has vote average is %.2f with %s voting.' % (row['name'], row['average'], row['cont']) )

## Here all-star cast movies for you

Sometimes you may want to watch a movie with many star actors! Easy to find those movies among thousands of entries! You need to `JOIN` your two maps on `movie id` and filter those with a total actor popularity point larger than 500. You can use `SUM()` function to find the total value of a column for a group, in this case, for every movie since we are grouping according to `title` of movie. You don't have to deal with any loops, control statements, or additional storage!</br>


In [ ]:
query = """
    SELECT m.original_title AS title, m.release_date AS r_date, SUM(a.popularity) AS point
    FROM movies m
    JOIN casts c ON m.id = c.movie_id
    JOIN actors a ON c.actor_id = a.id
    GROUP BY m.original_title, m.release_date
    HAVING SUM(a.popularity) > 150
    ORDER BY point DESC
"""

result = client.sql.execute(query).result()

for row in result:
        print(("%s %s %d") % (row['title'], row['r_date'], row['point']))

## Get the movies that get the highest review ratings from users

Now, we will try to get the top ten movies with average ratings higher than eight and at least reviewed by five different users. Note that `reviews` are separate map and different than `vote` which is a column of `movies` table. It looks complicated, right? Typically, you must traverse the two maps in order square. First, get all the reviews for a movie, calculate the average rating, store results in another data structure, and repeat it for all the movies. In the end, you must sort the results in descending order and get the highest ten movies.

Don't give up, we are there for you! Using the Hazelcast SQL support, you can use the power of SQL functions and aggregate operations like `ORDER BY()`, `LIMIT()`, and `AVG()` on your Map! You need to write your SQL query as a string and then execute it on your map with just one line, `sql.Execute()`. It's that easy! This operation returns a table, which consists of rows. You can traverse inside results by a simple `for` loop.

In [ ]:
query = """
    SELECT m.original_title as name, AVG(r.rating) as rating
    FROM movies m
    JOIN reviews r ON r.movie_id = m.id
    GROUP BY m.original_title
    HAVING AVG(r.rating) > 8 AND COUNT(*) > 5
    ORDER BY rating DESC
    LIMIT 10
"""

result = client.sql.execute(query).result()

for row in result:
        print("Movie Name: %s - Average Rating: %.2f" % (row['name'], row.get_object('rating')))

## Curious about user reviews?

There are many detailed reviews in our database. You can find a randomly selected entry for a movie or actor you are curious about. You can use the `LIKE` keyword for searching a specific keyword in our Map. For example, we searched for entries about Leanardo DiCaprio and got one between them randomly using the `RAND()` function. You can increase `LIMIT` to get more reviews and add multiple keywords. 

You can try the same thing with movie descriptions to find a specific movie informations, just use `movie.description` column!

In [ ]:
query = """
    SELECT m.original_title, r.author, r.rating, r.content
    FROM movies m, reviews r
    WHERE r.movie_id=m.id AND 
          r.content LIKE '%recommend%' AND 
          r.rating IS NOT NULL
    ORDER BY RAND()
    LIMIT 1
"""

result = client.sql.execute(query).result()

for row in result:
        print(("Movie name %s") % row.get_object('original_title'))
        print("-"*75)

        print("Review by %s rating is %s/10" % (row.get_object('author'), row.get_object('rating')))
        print("-"*75)
        print(textwrap.fill(row.get_object('content'), 75))

# Conculusion

Applications produces enormous amount of data every day. Most of them are wasted due to the high workload of processing, especially in real-time. As you can see from the examples, even if it is a simple map search, the algorithmic complexity and effort to execute the filter are tremendous.

Hazelcast offers a simple solution to evaluate and process all your data more easily and quickly as a real-time data processing platform. Using Hazelcast's SQL engine, you can skip all of the details and directly work on the value of your customer's data. You can infer much information without dealing with hundreds of line codes and slow executions.

You can follow our website and social media channels to learn about Hazelcast ecosystem more. We would love to answer your questions over our Community channels, find links here: https://hazelcast.com/dev-community/, also feel free to reach out at developer-relations@hazelcast.com.